In [1]:
# chargement des données de playlist
import numpy as np
data = np.load("./music_2.npy")
len(data)

100000

In [2]:
# récupération uniquement des identifiant de morceaux // suppression des identifiant d'artiste
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]

In [3]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

In [4]:
# nombre d'occurence de chaque morceaux ?
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;

In [5]:
# filtrage des morceaux très peu fréquents
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 7, playlist)) for playlist in playlist_track]
track_f = np.unique(np.concatenate(playlist_track_filter))
Vt = len(track_f)

In [6]:
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((track_f[i],i) for i in range(0, Vt))
# conversion des playlisat en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

In [7]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

Using TensorFlow backend.


In [8]:
# hyper-paramètres de word2vec :
# dimension de l'espace latent
vector_dim = 50
# taille de la fenêtre de voisinage
window_width = 5
# sur-échantillonage des exemples négatifs
neg_sample = 4.
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = -0.1

In [9]:
# comptage du nombre d'occurences des morceaux dans les playlist filtrées
tracks_counts_f = dict((track_f[i],0) for i in range(0, Vt))
for p in playlist_track_filter:
    for t in p:
        tracks_counts_f[t]=tracks_counts_f[t]+1;
# construction de la table de tirage des morceaux pour les exmeple negatif en utilisant ces fréquences
spt_tracks=np.array(list(map(lambda a:tracks_counts_f[a],track_f)),np.float)
sptn_tracks=np.power(spt_tracks,samp_coef)
sptn_tracks=sptn_tracks/np.sum(sptn_tracks)
sptn_tracks=np.cumsum(np.sort(sptn_tracks)[::-1])

In [20]:
# construction du réseau word2vec

# entrée deux entier (couple de morceaux)
input_target_t = Input((1,), dtype='int32')
input_context_t = Input((1,), dtype='int32')

# définition de l'embeding
embedding_t_t = Embedding(Vt, vector_dim, input_length=1, name='embedding_t')
# projection du premier morceau
target_t = embedding_t_t(input_target_t)
target_t = Reshape((vector_dim, 1))(target_t)

# projection du second morceaux
context_t = embedding_t_t(input_context_t)
context_t = Reshape((vector_dim, 1))(context_t)

# calcul de la sortie
dot_product_t = Dot(axes=0)([target_t, context_t])
dot_product_t = Reshape((1,))(dot_product_t)
output_t = Dense(1, activation='sigmoid',name="classif")(dot_product_t)

# definition du modèle
SkipGram_t = Model(inputs=[input_target_t, input_context_t], outputs=output_t)
SkipGram_t.compile(loss='binary_crossentropy', optimizer='adam')

In [21]:
# définition du générateur de couple de morceaux (y=0 <-> aléatoire, y=1 <-> proche dans une playlist)
import random
def track_ns_generator(corpus_num,nbm):
    while 1:
        Data=[]
        Labels=[]
        for i, doc in enumerate(random.sample(corpus_num,nbm)):
            data, labels = skipgrams(sequence=doc, vocabulary_size=Vt, window_size=window_width, negative_samples=neg_sample,sampling_table=sptn_tracks)
            if (len(data)>0):
                Data.append(np.array(data, dtype=np.int32))
                Labels.append(np.array(labels, dtype=np.int32))
        Data=np.concatenate(Data)
        Labels=np.concatenate(Labels)
        x=[Data[:,0],Data[:,1]]
        y=Labels
        yield (x,y)

In [22]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,10000)
index_app  = np.setdiff1d(range(100000),index_tst)

play_app   = [corpus_num_track[i] for i in index_app]
play_tst  = [corpus_num_track[i] for i in index_tst]


In [23]:
# apprentissage
hist=SkipGram_t.fit_generator(track_ns_generator(play_app,min_batch_size),200,50)

Epoch 1/50
200/200 [==============================] - 5s 26ms/step - loss: 0.6651
Epoch 2/50
200/200 [==============================] - 5s 23ms/step - loss: 0.6125
Epoch 3/50
200/200 [==============================] - 5s 23ms/step - loss: 0.5500
Epoch 4/50
200/200 [==============================] - 5s 23ms/step - loss: 0.5026
Epoch 5/50
200/200 [==============================] - 5s 23ms/step - loss: 0.4743
Epoch 6/50
200/200 [==============================] - 5s 24ms/step - loss: 0.4482
Epoch 7/50
200/200 [==============================] - 5s 23ms/step - loss: 0.4324
Epoch 8/50
200/200 [==============================] - 5s 23ms/step - loss: 0.4109
Epoch 9/50
200/200 [==============================] - 5s 23ms/step - loss: 0.3951: 0s - loss: 0.39
Epoch 10/50
200/200 [==============================] - 5s 24ms/step - loss: 0.3832
Epoch 11/50
200/200 [==============================] - 5s 24ms/step - loss: 0.3692
Epoch 12/50
200/200 [==============================] - 5s 23ms/step - loss: 0.3

In [24]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = SkipGram_t.get_weights()[0]

In [25]:
from sklearn.metrics import pairwise_distances

In [220]:
# fonction retpournant les indices des s plus proches voisins des seeds dans la matrice X
# a vous de jouer avec pairwise_distances, np.min, np.argsort,...
def predict(seeds,s,X):
    V = X.shape[0]
    others = np.setdiff1d(range(V),seeds)
    D = pairwise_distances(X[seeds,:],X[others,:],'cosine')
    return others[np.argsort(np.min(D,0))[:s]]

In [221]:
# exemple pour 4 suggestions avec les 2 premiers morceaux en seeds 
pr=predict([0,1],4,vectors_tracks)
pr

array([10131,  6334,     2,     5])

In [115]:
# estimation des performances
# hit@10
# nombre de bonne prediction
goodpred = []
# nombre de predictions faites
nbpred   = []
# pour chaque playlist
for p in play_tst:
    # si au moins deux chansons
    if (len(p)>1):
        # recuperations des seeds 5 premiers morceaux ou moins si la playlist contient moins de 5 morceaux
        seeds  = p[:np.min([5,len(p)-1])]
        # recuperations de la suite de la playlist que nous allons comparer à nos suggestions
        topred = p[np.min([5,len(p)-1]):]
        # construction des suggestions 10 suggestions par morceaux a predire
        prediction = predict_opt(seeds,10*len(topred),vectors_tracks,kdt)
        # comptage du nombre de morceaux présent dans nos suggestions
        goodpred.append(len(np.intersect1d(prediction,topred)))
        # stockage du nombre de predictions
        nbpred.append(len(topred))
# proportions de morceux présents dans nos suggestions
hitat10 = np.sum(goodpred)/np.sum(nbpred)

In [116]:
hitat10

0.2011281061029524

# A faire

- faire quelque graphique sen particulier pour discuter le choix de n'avoir garder que les morceaux présents dans plus de sept playlists
- coder la fonction predict permetant de trouver les morceaux les plus proches des seeds (dans un premier temps de manière simple en utilisant la fonction pairwise_dist de sklearn)
- tester l'influence des différents type de distance sur les les performances en terme de hit@10
- vous pourrez ensuite essayé d'optimiser cette recherche https://scikit-learn.org/stable/modules/neighbors.html
- faites varier un hyper-paramètres sur une plage raisonable 
- faire un graphique de hit@10 en fonction des valeurs de l'hyperparamètre
- faire de même avec un second
- proposer des valeures finales pour les hyper-paramètres
- joindre avec les meta-données, url des morceaux ....
- exporter // faire le liens avec le front-end

## Aide

In [209]:
# aide jointure
import pandas as pd
tr_meta=pd.read_csv("./Tracks.csv")
jdf = pd.DataFrame({"id":track_f,"index":range(Vt)})
jdf["deezer_id"]=jdf["id"].apply(lambda x: float(x.split("_")[1]))

In [210]:
trj_meta = tr_meta.merge(jdf, left_on="id",right_on="deezer_id")
trj_meta.set_index("index",inplace=True)

In [211]:
trj_meta

,Unnamed: 0,album,artist,available_countries,bpm,contributors,disk_number,duration,error,explicit_lyrics,...,readable,release_date,share,title,title_short,title_version,track_position,type,id_y,deezer_id
index,,,,,,,,,,,,,,,,,,,,,
0,0,"{u'title': u'Alone', u'type': u'album', u'rele...",{u'picture': u'http://api.deezer.com/artist/79...,"[AE, AF, AG, AI, AL, AM, AN, AO, AQ, AR, AS, A...",116.0,[{u'picture': u'http://api.deezer.com/artist/7...,1.0,273.0,NaN,False,...,True,2015-05-25,https://www.deezer.com/track/100001884?utm_sou...,Alone,Alone,NaN,1.0,track,track_100001884,100001884.0
1,1,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",103.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,239.0,NaN,False,...,False,2015-05-15,https://www.deezer.com/track/100004586?utm_sou...,It Was Always You,It Was Always You,NaN,3.0,track,track_100004586,100004586.0
2,2,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",130.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,238.0,NaN,False,...,False,2015-05-15,https://www.deezer.com/track/100004588?utm_sou...,Unkiss Me,Unkiss Me,NaN,4.0,track,track_100004588,100004588.0
3,3,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",119.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,235.0,NaN,True,...,False,2015-05-15,https://www.deezer.com/track/100004590?utm_sou...,Sugar,Sugar,NaN,5.0,track,track_100004590,100004590.0
4,4,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",97.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,203.0,NaN,False,...,False,2015-05-15,https://www.deezer.com/track/100004592?utm_sou...,Leaving California,Leaving California,NaN,6.0,track,track_100004592,100004592.0
5,5,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",126.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,219.0,NaN,True,...,False,2015-05-15,https://www.deezer.com/track/100004594?utm_sou...,In Your Pocket,In Your Pocket,NaN,7.0,track,track_100004594,100004594.0
6,6,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",164.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,196.0,NaN,True,...,False,2015-05-15,https://www.deezer.com/track/100004596?utm_sou...,New Love,New Love,NaN,8.0,track,track_100004596,100004596.0
7,7,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",114.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,226.0,NaN,False,...,False,2015-05-15,https://www.deezer.com/track/100004598?utm_sou...,Coming Back For You,Coming Back For You,NaN,9.0,track,track_100004598,100004598.0
8,8,"{u'title': u'V (Deluxe)', u'type': u'album', u...",{u'picture': u'http://api.deezer.com/artist/11...,"[AE, AF, AG, AI, AL, AM, AO, AQ, AR, AS, AT, A...",122.0,[{u'picture': u'http://api.deezer.com/artist/1...,1.0,194.0,NaN,True,...,False,2015-05-15,https://www.deezer.com/track/100004600?utm_sou...,Feelings,Feelings,NaN,10.0,track,track_100004600,100004600.0


In [224]:
#aide predict_opt
#créer un kd-tree
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=30, metric='euclidean')
#fonction de recherche a compléter
#vérifer la cohérences des résulats avec predict
def predict_opt(seeds,s,X,kdt):
     return 0

In [ ]:
# vérification de la cohérences des résultats des suggestion

In [217]:
trj_meta.sort_values(by=['rank'],ascending=False)["title"].head(20)

index
6346                                     Shape of You
9152                  I Will Survive (Single Version)
7090                                      Look At Me!
20466    I Will Survive (Lalala), Original Radio Edit
11875                                           IDGAF
7215                                          Perfect
7131                         Something Just Like This
20170           Magic In The Air (feat. Ahmed Chawki)
7500                                        Despacito
6458                                            Faded
15120                                  Jocelyn Flores
11879                                       New Rules
3043                                        One Dance
13197                    De Trás Pra Frente (Ao Vivo)
14386        Meant to Be (feat. Florida Georgia Line)
12572                                       Me Rehúso
15126                                       Fuck Love
15122              Everybody Dies In Their Nightmares
11523                 

In [222]:
# les voisins de i will survive ?
b=predict([9152],4,vectors_tracks)
trj_meta.loc[b,"title"]

index
21843                              Hot Stuff
19030           YMCA (Original Version 1978)
16915                             Daddy Cool
19029    In the Navy (Original Version 1979)
Name: title, dtype: object

In [216]:
# un peu de musique
import IPython
IPython.display.Audio(trj_meta.loc[19030,"preview"])